Import Libraries

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

Load Processed Data

In [10]:
# بارگیری داده‌های پردازش‌شده
orders = pd.read_csv(r"E:\data analysis project\InstacartMarketBasketAnalysisPortflio\data\processed\cleaned_orders.csv")
user_features = pd.read_csv(r"E:\data analysis project\InstacartMarketBasketAnalysisPortflio\data\processed\user_features.csv")
product_features = pd.read_csv(r"E:\data analysis project\InstacartMarketBasketAnalysisPortflio\data\processed\product_features.csv")

In [13]:
print(orders.columns.tolist())

['order_id', 'user_id', 'product_id', 'reordered']


Feature Engineering for Users

In [14]:
# تعداد سفارشات هر کاربر (Total Orders)
user_features['total_orders'] = orders.groupby('user_id')['order_id'].nunique().reset_index(drop=True)

# تعداد محصولات سفارش‌داده‌شده توسط هر کاربر (Total Products Ordered)
user_features['total_products_ordered'] = orders.groupby('user_id')['product_id'].count().reset_index(drop=True)

# میانگین تعداد محصولات در هر سفارش (Avg Basket Size)
user_features['avg_basket_size'] = user_features['total_products_ordered'] / user_features['total_orders']


Feature Engineering for Products

In [15]:
# تعداد سفارشات هر محصول (Order Count)
product_features['order_count'] = orders.groupby('product_id')['order_id'].count().reset_index(drop=True)

# تعداد سفارش‌های مجدد هر محصول (Reordered Count)
product_features['reordered_count'] = orders[orders['reordered'] == 1].groupby('product_id')['order_id'].count().reset_index(drop=True)

# نرخ سفارش مجدد (Reorder Rate)
product_features['reorder_rate'] = product_features['reordered_count'] / product_features['order_count']

Merge All Features

In [16]:
# ادغام سفارشات با ویژگی‌های کاربر
merged_data = pd.merge(orders, user_features, on='user_id', how='left')

# ادغام نتیجه با ویژگی‌های محصول
merged_data = pd.merge(merged_data, product_features, on='product_id', how='left')

Final Feature Selection

In [17]:
# انتخاب ستون‌های مرتبط برای مدل‌سازی
model_ready_data = merged_data[[
    'user_id', 
    'product_id', 
    'total_orders', 
    'total_products_ordered', 
    'avg_basket_size', 
    'order_count', 
    'reorder_rate', 
    'reordered'
]]